In [21]:
! pip install beautifulsoup4
! pip install requests

In [29]:
from bs4 import BeautifulSoup
import requests

url_link = "https://mechanicalkeyboards.com/shop/index.php?l=product_list&c=1"

In [30]:
response = requests.get(url_link)
html = response.text
soup = BeautifulSoup(html,'html.parser')


In [47]:
product_soup = soup.select("div.product-name a")
image_link = soup.select("div.imagebox li a img")
full_image_url = "https://mechanicalkeyboards.com/shop/" + image_link[0]['src']

product_name = product_soup[0].text
product_url = product_soup[0]['href']

print(f"Name:{product_name}\nUrl:{product_url}\nImage Url: {full_image_url}")



Name:Ducky One 2 Mini Pure White - RGB LED 60% Double Shot PBT Mechanical Keyboard
Url:https://mechanicalkeyboards.com/shop/index.php?l=product_detail&p=5027
Image Url: https://mechanicalkeyboards.com/shop/images/products/thumb_5027_thumb_DKON2061ST-USPDWWT7UJ_main.png


In [59]:

for (name,url) in zip(product_soup, image_link):
    product_name = name.text
    product_url = name['href']
    full_image_url = "https://mechanicalkeyboards.com/shop/" + url['src']
    print(f"Name:{product_name}\nUrl:{product_url}\nImage Url: {full_image_url}\n")

Name:Ducky One 2 Mini Pure White - RGB LED 60% Double Shot PBT Mechanical Keyboard
Url:https://mechanicalkeyboards.com/shop/index.php?l=product_detail&p=5027
Image Url: https://mechanicalkeyboards.com/shop/images/products/thumb_5027_thumb_DKON2061ST-USPDWWT7UJ_main.png

Name:Ducky One 2 Mini v2 RGB LED  60% Double Shot PBT Mechanical Keyboard
Url:https://mechanicalkeyboards.com/shop/index.php?l=product_detail&p=4322
Image Url: https://mechanicalkeyboards.com/shop/images/products/thumb_5027_thumb_DKON1861ST-USPDWWT1_2.jpg

Name:Ducky One 2 SF RGB LED  65% Double Shot PBT Mechanical Keyboard
Url:https://mechanicalkeyboards.com/shop/index.php?l=product_detail&p=5461
Image Url: https://mechanicalkeyboards.com/shop/images/products/thumb_5027_thumb_DKON1861ST-USPDWWT1_3.jpg

Name:Ducky Mecha Mini v2 RGB LED  60% Double Shot PBT Mechanical Keyboard
Url:https://mechanicalkeyboards.com/shop/index.php?l=product_detail&p=5593
Image Url: https://mechanicalkeyboards.com/shop/images/products/thumb_5